In [8]:
from utils import llms
from utils import prompts
import yaml
import jsonlines
import json
import toml
import time
import tqdm

from together import Together

# # open config
# config_path = "./configs/config_zero-shot.yaml"
# with open(config_path, 'r') as f:
#     content = f.read()
# config = yaml.safe_load(content)

input_file = "/Users/jjr/PycharmProjects/RARR-rep/input/processed/fava_input_processed_2.jsonl"
output_file = "./output/fava_zero_shot_output_dsr1_2.jsonl"
# model="flywheel25/meta-llama/Llama-3.2-1B-Instruct-7695e079"
# model="flywheel25/meta-llama/Meta-Llama-3.1-8B-Instruct-Reference-38ef9c4e"
# model="flywheel25/meta-llama/Meta-Llama-3.1-70B-Instruct-Reference-bbae0ef2"
model="deepseek-ai/DeepSeek-R1"

# instantiate llm
# llm = llms.instantiate_llm(config)
api_key = ""
client = Together(api_key=api_key)


# continue_flag = False  # or read it from some config or CLI argument
# mode = "a" if continue_flag else "w"
# continue_idx = 2403
mode = 'w'
# load input data
# iterate over the input sentences
with jsonlines.open(input_file) as input_reader, \
        open(output_file, mode, encoding="utf-8") as output_writer:
    for i, item in enumerate(tqdm.tqdm(input_reader), start=1):
 
        # if i < continue_idx:
        #     continue
        
        # start timing the iteration
        start_time = time.perf_counter()

        output_data = item

        sentence = output_data["decon_sentence"]

        # Populate the prompt with the required data
        prompt_data = {"sentence": sentence, "example_json": "{\"sentence_score\": 1, \"query_score\": 0}"}

        # load prompt config
        path_to_config = f"./prompts/config_zero-shot-1_Llama.toml"

        try:
            with open(path_to_config) as f:
                prompt_config = toml.load(f)
        except FileNotFoundError:
            print("File not found.")
            exit()
 

        # create llm messages
        messages = prompts.create_llm_messages(prompt_config, prompt_data)
        
        # prompt llm
        # response = llm.generate(messages)

        response = client.chat.completions.create(model=model, messages=messages)
 
        # parse response
        response = response.choices[0].message.content

        data = {}
        for line in response.splitlines():
            if "=" in line:
                key, value = line.split("=", 1)
            elif ":" in line:
                key, value = line.split(":", 1)
            else:
                continue
            data[key.strip()] = value.strip()


        # Convert numeric values from strings to integers if necessary
        try:
            has_facts = int(data.get("has_facts", 0))
        except ValueError:
            print("error parsing has_facts")
            has_facts = 0
        try:
            has_errors = int(data.get("has_errors", 0))
        except ValueError:
            print("error parsing has_errors")
            has_errors = 0
        reason = data.get("reason", "")
        data = {'response': response, 'has_facts': has_facts, 'has_errors': has_errors, 'reason': reason}

        iteration_time = time.perf_counter() - start_time
        output_data["zs_iteration_time"] = iteration_time

        # add queries to output data
        output_data["zs_response"] = data

        # write 1 line of output
        output_writer.write(json.dumps(output_data, ensure_ascii=False) + "\n")
        output_writer.flush()



2154it [2:38:47,  4.42s/it]
